In [18]:
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, LSTM, Masking, Dropout, TimeDistributed, Input
from tensorflow.keras.models import Sequential



In [19]:
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available: 1


In [20]:
def padding(frames, max_frames):
    new_frames = frames
    while len(new_frames) < max_frames:
        zero_array = np.zeros((224,224, 3))
        new_frames.append(zero_array)
    
    return new_frames

In [21]:
def extract_and_resize_frames(video_path, target_size=(224,224), max_frames=120):
    cap = cv2.VideoCapture(video_path)
    frames = []
    
    while len(frames) < max_frames:
        ret, frame = cap.read()
        if not ret:
            break
    
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame_resized = cv2.resize(frame, target_size)
        frame_resized = frame_resized / 255.0
        frames.append(frame_resized)
    
    #while len(frames) < max_frames:
    #    frames.append(np.zeros((640, 360)))

    frames = padding(frames, 120)
    
    cap.release()
    return np.array(frames)

In [22]:
def create_padding_mask(frames, max_frames=120):
    mask = np.ones((max_frames,))  
    if len(frames) < max_frames:
        mask[len(frames):] = 0  
    return mask

In [23]:
def repeat_rows(df, num_repeats=120):
    repeated_df = pd.DataFrame(np.tile(df.values, (num_repeats, 1)), columns=df.columns)
    
    return repeated_df

In [24]:
def generate_df(X_folder, y_filepath):
    X_filepaths = []
    X_arrays = []
    
    for i in range(35):
        X_filename = f"{X_folder}{i+1}.mp4"
        X_filepaths.append(X_filename)
    
    for filepath in X_filepaths:
        X_array = extract_and_resize_frames(filepath)
        X_arrays.append(X_array)

    y_df = pd.read_csv(y_filepath)
    y_df.drop(columns='filename', axis=1, inplace=True)
    y_df = y_df.head(35)
    
    y_df = repeat_rows(y_df)

    X_arrays = np.array(X_arrays)   
    y_array = y_df.to_numpy()
    
    return X_arrays, y_array

In [25]:
X, y = generate_df("../app/processed_data/", "../model/data/coordinate_data.csv")
X.shape


(35, 120, 224, 224, 3)

In [26]:
def create_feature_extraction_cnn(input_shape):
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)
    
    for layer in base_model.layers:
        layer.trainable = False
        
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(256, activation='relu')
    ])
    
    return model

In [27]:
def feature_extracter(X):
    all_features = []
    model = create_feature_extraction_cnn((224, 224, 3))
    model.save_weights('CNN.weights.h5')
    model.save('full_CNN.keras')
    for video in X:
        features = model.predict(video)
        all_features.append(features)
    
    
    
    all_features = np.array(all_features)
    return(all_features)

   


In [28]:
X = feature_extracter(X)
X.shape

4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 686ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
4/4 ━━━━━━━

(35, 120, 256)

In [29]:
def build_model(input_shape, X, y):
    y = y.reshape((35,120,2))

    model = Sequential([
        Masking(mask_value=0.0, input_shape=input_shape),
        LSTM(256, return_sequences=True),
        Dropout(0.3),
        LSTM(128, return_sequences=True),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dense(2)

        
    ])
    
    model.compile(
        optimizer='adam', 
        loss='mse',
        metrics = ['mae']
    )
    
    model.fit(X, y, epochs=25)
    
    return model

In [30]:
model = build_model(input_shape=(120, 256), X=X, y=y)

Epoch 1/25


/home/saahas/.pyenv/versions/3.11.11/envs/virtualenv/lib/python3.11/site-packages/keras/src/layers/core/masking.py:48: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 6.5872 - mae: 1.7063
Epoch 2/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 3.2948 - mae: 1.3776
Epoch 3/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 3.3246 - mae: 1.4485
Epoch 4/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 3.1272 - mae: 1.3382
Epoch 5/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 3.0970 - mae: 1.3102
Epoch 6/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 3.1426 - mae: 1.3222
Epoch 7/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 3.0913 - mae: 1.2958
Epoch 8/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 3.0559 - mae: 1.2958
Epoch 9/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 3.0505 - mae: 1.3144
Epoch 10/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 3.0626 - mae: 1.3203
Epoch 11/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 3.0521 - mae: 1.3193
Epoch 12/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 3.0425 - mae: 1.3120
Epoch 13/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 3.0276 - mae: 

In [31]:
model.save_weights('LSTM.weights.h5')
model.save('full_LSTM.keras')

In [32]:
#how to load model into API
'''
from tensorflow.keras.models import load_model
model = load_model('full_model.keras')
model.load_weights('model_weights.h5')
'''

"\nfrom tensorflow.keras.models import load_model\nmodel = load_model('full_model.keras')\nmodel.load_weights('model_weights.h5')\n"